In [4]:
import numpy as np
import os
import h5py
import time
import copy
from random import randint
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

C:\Users\wangwang\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


cuda:0


In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            #nn.MaxPool2d(2, 2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            #nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Dropout(p=0.5, inplace=False))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            #nn.MaxPool2d(2, 2))
            
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            #nn.BatchNorm2d(64))
            nn.MaxPool2d(2, 2),
            nn.Dropout(p=0.5, inplace=False))
            
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            #nn.MaxPool2d(2, 2))
            
        self.layer6 = nn.Sequential(
            nn.Conv2d(64, 64, 3,stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False))
            
        self.layer7 = nn.Sequential(
            nn.Conv2d(64, 64, 3,stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            #nn.MaxPool2d(2, 2))
            
        self.layer8 = nn.Sequential(
            nn.Conv2d(64, 64, 3,stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.5, inplace=False))
            
        
        
        self.fc1 = nn.Linear(1024, 120)  
        self.fc2 = nn.Linear(120, 10)
        #self.linear3 = nn.Linear(layer_two_out, 10)
        #self.softmax = nn.LogSoftmax(dim=1)

        
    def forward(self, x):
        #print(x.size())
        out = self.layer1(x)
        #print(out.size())
        out = self.layer2(out)
        #print(out.size())
        out = self.layer3(out)
        #print(out.size())
        #out = out.view(-1, 64 * 18 * 18)
        
        out = self.layer4(out)
        out = self.layer5(out)
        #print(out.size())
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        #print(out.size())
        #out = out.reshape(out.size(0), -1)
        #print(out.size())
        out = out.view(-1, out.shape[1] * out.shape[2] * out.shape[3])
        #print(out.size())
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = out.softmax(1)
        #out = self.linear3(out)
        #out = self.softmax(out)
        
        return out

model = Model().to(device)

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64,4,1,2)
        self.pool = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 4,1,2)
        self.drop = nn.Dropout(p = 0.5)
        self.fc1 = nn.Linear(64*18*18, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #layer1
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        #layer2
        x = self.pool(F.relu(self.conv2(x)))
        x = self.drop(x)
        #layer3
        x = F.relu(self.conv2(x))
        x = self.bn1(x)
        #print(x.size())
        
        x = x.view(-1, 64*18*18)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model1 = Model1().to(device)

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)

In [9]:
#  TRAINING
epoch =5
for epoch in range(epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.571
[1,  4000] loss: 0.575
[1,  6000] loss: 0.609
[1,  8000] loss: 0.628
[1, 10000] loss: 0.603
[1, 12000] loss: 0.612
[2,  2000] loss: 0.492
[2,  4000] loss: 0.515
[2,  6000] loss: 0.541
[2,  8000] loss: 0.527
[2, 10000] loss: 0.544
[2, 12000] loss: 0.539
[3,  2000] loss: 0.438
[3,  4000] loss: 0.465
[3,  6000] loss: 0.488
[3,  8000] loss: 0.473
[3, 10000] loss: 0.478
[3, 12000] loss: 0.499
[4,  2000] loss: 0.385
[4,  4000] loss: 0.410
[4,  6000] loss: 0.435
[4,  8000] loss: 0.423
[4, 10000] loss: 0.434
[4, 12000] loss: 0.465
[5,  2000] loss: 0.360
[5,  4000] loss: 0.366
[5,  6000] loss: 0.384
[5,  8000] loss: 0.390
[5, 10000] loss: 0.382
[5, 12000] loss: 0.386
Finished Training


In [10]:
#correct = 0
#total = 0
#with torch.no_grad():
#    for data in testloader:
#        inputs, labels = inputs.to(device), labels.to(device)
#        outputs = model(inputs)
#        _, predicted = torch.max(outputs.data, 1)
#        total += labels.size(0)
#        correct += (predicted == labels).sum().item()

#print('Accuracy of the network on the 10000 test images: %d %%' % (
#    100 * correct / total))

#model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 70.89 %
